In [ ]:
import pickle
import pandas as pd

p1 = "/home/jonfrey/results/rpose/evaluate/full_evaluation/results.pkl"
p2 = "/home/jonfrey/results/rpose/evaluate_pose/2021-04-24T10:19:18_tracking_with_gt_pose/results.pkl"
with open(p1 , 'rb') as handle:
    res = pickle.load(handle)

pred_auc_add_s = { k:round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADD_S') != -1 and k.find('PRED') != -1)}
init_auc_add_s = { k:round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADD_S') != -1 and k.find('INIT') != -1)}

pred_auc_add_s
init_auc_add_s

pred_auc_add_s_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADD_S') != -1 and k.find('PRED') != -1)]
init_auc_add_s_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADD_S') != -1 and k.find('INIT') != -1)]

pred_auc_adds_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADDS') != -1 and k.find('PRED') != -1)]
init_auc_adds_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('AUC_ADDS') != -1 and k.find('INIT') != -1)]

pred_2cm_add_s_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('2CM_ADD_S') != -1 and k.find('PRED') != -1)]
init_2cm_add_s_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('2CM_ADD_S') != -1 and k.find('INIT') != -1)]

pred_2cm_adds_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('2CM_ADDS') != -1 and k.find('PRED') != -1)]
init_2cm_adds_ls = [ round(v,2) for (k,v) in res.items() if ( k.find('2CM_ADDS') != -1 and k.find('INIT') != -1)]

d = {'IDX':list(range(21)), 
     'AUC_ADD_S_INIT': init_auc_add_s_ls, 'AUC_ADD_S_PRED': pred_auc_add_s_ls,
    'AUC_ADDS_INIT': init_auc_adds_ls, 'AUC_ADDS_PRED': pred_auc_adds_ls
    }
df = pd.DataFrame(data=d)
df


In [ ]:
from pytorch_lightning.loggers.neptune import NeptuneLogger
import os
logger = NeptuneLogger(
          api_key=os.environ["NEPTUNE_API_TOKEN"],
          project_name="jonasfrey96/rpose",
          experiment_id='RPOS-340',
          close_after_fit = False,
        )
logger.experiment

d = logger.experiment.get_channels()
ar = []
for e in d.keys():
    try:  
        ar.append( logger.experiment.get_numeric_channels_values( e ) ) 
    except:
        pass
dic = {}
for data in ar:
    k = str( data.keys()[1] ) 
    dic[k] =  data[k].to_numpy()

In [ ]:
_add_s = {k:v.tolist() for (k,v) in dic.items() if (str(k).find("add_s_obj")  != -1 and str(k).find("_step")  != -1 )} 
_adds = {k:v.tolist() for (k,v) in dic.items() if (str(k).find("adds_obj")  != -1 and str(k).find("_step")  != -1  )}
list(_add_s.keys())

In [ ]:
with open(f'cfg/datasets/ycb/train.pkl', 'rb') as handle:
      mappings = pickle.load(handle)
      _names_idx = mappings['names_idx']
      _idx_names = mappings['idx_names']
        


In [ ]:
import os
import sys 
import numpy as np
os.chdir(os.path.join(os.getenv('HOME'), 'RPOSE'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/core'))
sys.path.append(os.path.join(os.getcwd() + '/segmentation'))
from pose_estimation import full_pose_estimation, compute_auc, compute_percentage


adds_auc = {}
add_s_auc = {}
get_obj = lambda x: int( x[x.find("obj")+3 : x.find("_step")] )
k1 = list( _adds.keys() )
k1.sort(key= get_obj )
k2 = list( _add_s.keys() )
k2.sort(key= get_obj )



for index_key in k1:
    adds_auc[ str(get_obj(index_key)) ] = round( compute_auc( np.array( _adds[index_key] )),2)
for index_key in k2:
    add_s_auc[ str( get_obj(index_key) ) ] = round( compute_auc( np.array(_add_s[index_key] )) ,2)

dicta = { 'IDX': list(add_s_auc.keys() ), 'Name': list( _idx_names.keys() ), 'ADD-(S)':  list(add_s_auc.values() ),  'ADDS':  list(adds_auc.values() )}

pd.DataFrame.from_dict( dicta ) # , columns= ['obj', 'val'])

In [ ]:
# PUTS ALL DATA INTO PROPERTIES



properties = {}
for index_key in _adds.keys():
  if len( _adds[index_key] ) > 0:
    properties['MEAN_ADDS__OBJ('+index_key+")_PRED_CM" ] = round( sum( _adds[index_key]) / len(_adds[index_key] ) *100,2)
for index_key in _add_s.keys():
  if len( _add_s[index_key] ) > 0:        
    properties['MEAN_ADD_S_OBJ('+index_key+")_PRED_CM" ] = round( sum( _add_s[index_key]) / len( _add_s[index_key] ) *100,2)

for index_key in _adds.keys():
  if len( _adds[index_key] ) > 0:
    properties['2CM_ADDS_OBJ('+index_key+")_PRED"] = round( compute_percentage( np.array( _adds[index_key] ))  ,2)
for index_key in _add_s.keys():
  if len(_add_s[index_key] ) > 0:
    properties['2CM_ADD_S_OBJ('+index_key+")_PRED"] = round( compute_percentage( np.array( _add_s[index_key] ))  ,2)

kk = list( properties.keys())
kk.sort()


In [ ]:
ks = [ k for k  in list(d.keys()) if (str(k).find("adds_obj")  != -1 and str(k).find("_step")  != -1  ) ]
ks.sort(key=lambda x: int( x[x.find("obj")+3 : x.find("_step")] ) )
logger.experiment.get_numeric_channels_values( *ks )